### Read the mergertree data, and compute extra properties from the particle data.

This requires the raw snapshot data, and is not done at every output. This script handles one snapshot at a time (as it can take several hours and > 10 GB of memory). Simply change the "snapshot variable" to run on a different snapshot.

### Define path for local modules

In [1]:
import sys
print(sys.path)

sys.path.append('/users/sawalati/.local/lib/python3.10/site-packages')

print("now:", sys.path)


['/users/sawalati', '/CSC_CONTAINER/miniconda/envs/env1/lib/python310.zip', '/CSC_CONTAINER/miniconda/envs/env1/lib/python3.10', '/CSC_CONTAINER/miniconda/envs/env1/lib/python3.10/lib-dynload', '', '/CSC_CONTAINER/miniconda/envs/env1/lib/python3.10/site-packages', '/CSC_CONTAINER/miniconda/envs/env1/lib/python3.10/site-packages/IPython/extensions', '/users/sawalati/.ipython']
now: ['/users/sawalati', '/CSC_CONTAINER/miniconda/envs/env1/lib/python310.zip', '/CSC_CONTAINER/miniconda/envs/env1/lib/python3.10', '/CSC_CONTAINER/miniconda/envs/env1/lib/python3.10/lib-dynload', '', '/CSC_CONTAINER/miniconda/envs/env1/lib/python3.10/site-packages', '/CSC_CONTAINER/miniconda/envs/env1/lib/python3.10/site-packages/IPython/extensions', '/users/sawalati/.ipython', '/users/sawalati/.local/lib/python3.10/site-packages']


### Imports

In [2]:
import numpy as np
import pynbody

import math

from scipy import spatial

import timeit
import glob
from tqdm import tqdm

import os
import pickle
import psutil

from astropy import units as u
from astropy.cosmology import FlatLambdaCDM
from astropy.cosmology import z_at_value

print ("loaded modules")

loaded modules


### Parameters

In [3]:
cosmo = FlatLambdaCDM(H0=67.74, Om0=0.3, Tcmb0=2.725)

hubble = .6774

set_LG = 'true'
NFiles = 512  # Number of tree files used

NSnapFiles = 400 # Number of Snapshot files used

snapshot = 15
snap_index = 50 - snapshot

treefile ='particle_tree.npz'
orbitsfile = 'orbits_savgol.npz'

snapfile_root = "/scratch/pjohanss/sawalati/mini_uchuu_snapshots/MiniUchuu_"  

boxSize = 400. / hubble * 1000. + 0.01 # kpc  # MiniUchuu. Still need to apply the scale factor

shell_radii_r = np.linspace(0.1,2.,39)  # fraction of separation
shell_radii_kpc = np.linspace(100, 2000, 39) # fixed distance

### Restore Tree data from File

In [4]:
with np.load(treefile, allow_pickle=True) as treedata:
    x_MW = treedata['list_x_MW']
    y_MW = treedata['list_y_MW']
    z_MW = treedata['list_z_MW']
    
    x_M31 = treedata['list_x_M31']
    y_M31 = treedata['list_y_M31']
    z_M31 = treedata['list_z_M31']

N_LG = len(x_MW)
    
print ("N_LG", N_LG)

x_MW = np.array(list(x_MW[i][snap_index] for i in range(N_LG)))
y_MW = np.array(list(y_MW[i][snap_index] for i in range(N_LG))) 
z_MW = np.array(list(z_MW[i][snap_index] for i in range(N_LG)))

x_M31 = np.array(list(x_M31[i][snap_index] for i in range(N_LG)))
y_M31 = np.array(list(y_M31[i][snap_index] for i in range(N_LG)))
z_M31 = np.array(list(z_M31[i][snap_index] for i in range(N_LG)))

print ("read Treedata from file")

N_LG 4902
read Treedata from file


### Read the list of Uchuu scalefactors

In [5]:
f = open('Uchuu_redshift_scalefactors.txt', 'r')

lines = f.readlines()
scale = []
                  
for line in lines:
    columns = line.split()
    scale.append(float(columns[2]))

scale = np.flip(scale)
print ("Read scalefactors")

Read scalefactors


### Define comoving radii

In [6]:
shell_radii_com = shell_radii_kpc*scale[snap_index]  # fixed comoving distance

### Loop over LG analogues and read the snapshots, file by file

In [7]:
print ("scale factor:", scale[snap_index])

print("Memory used", psutil.Process(os.getpid()).memory_info().rss / 1024 ** 2, " MB")

print ("radii (r):", shell_radii_r)
print ("radii (kpc):", shell_radii_kpc)

LG_x = 0.5*(x_MW + x_M31) * 1000 / hubble * scale[snap_index] # physical kpc
LG_y = 0.5*(y_MW + y_M31) * 1000 / hubble * scale[snap_index]
LG_z = 0.5*(z_MW + z_M31) * 1000 / hubble * scale[snap_index]

LG_r = ( (x_MW-x_M31)**2 + (y_MW-y_M31)**2 + (z_MW-z_M31)**2 )**.5 * 1000 / hubble * scale[snap_index]

boxSize = 400. / hubble * 1000. * scale[snap_index] + 0.01 # kpc  # MiniUchuu. Still need to apply the scale factor

print ("box", boxSize)

scale factor: 0.216898
Memory used 170.9453125  MB
radii (r): [0.1  0.15 0.2  0.25 0.3  0.35 0.4  0.45 0.5  0.55 0.6  0.65 0.7  0.75
 0.8  0.85 0.9  0.95 1.   1.05 1.1  1.15 1.2  1.25 1.3  1.35 1.4  1.45
 1.5  1.55 1.6  1.65 1.7  1.75 1.8  1.85 1.9  1.95 2.  ]
radii (kpc): [ 100.  150.  200.  250.  300.  350.  400.  450.  500.  550.  600.  650.
  700.  750.  800.  850.  900.  950. 1000. 1050. 1100. 1150. 1200. 1250.
 1300. 1350. 1400. 1450. 1500. 1550. 1600. 1650. 1700. 1750. 1800. 1850.
 1900. 1950. 2000.]
len LG_x 4902
LG_x [  4212.63208345  11345.35835475   8171.37599647 ... 121813.11711814
 123435.55994559 116983.48482941]
len LG_r 4902
box 128076.77409802184


### Extract particles from the files on disk. This can take several hours per snapshot

In [8]:
# count the number of particles

LG_NP_kpc = np.zeros((N_LG, len(shell_radii_kpc)), dtype=int)
LG_NP_r = np.zeros((N_LG, len(shell_radii_r)), dtype=int)
LG_NP_com = np.zeros((N_LG, len(shell_radii_com)), dtype=int)

## Loop over the snapshot files

for filenum in range (NSnapFiles):
    print ("file ", filenum+1, ' of ', NSnapFiles)
    
    s = 0
    s = pynbody.load(snapfile_root+'0'+str(snapshot)+'.gad.'+str(filenum))

    print("Memory used", psutil.Process(os.getpid()).memory_info().rss / 1024 ** 2, " MB")
    
    pos = 0
    pos = s['pos'] / hubble * 1000 * scale[snap_index] # physical kpc
    
    print ("min / max x:", min(pos[:,0]), " / ", max(pos[:,0]))
    print ("min / max y:", min(pos[:,1]), " / ", max(pos[:,1]))
    print ("min / max z:", min(pos[:,2]), " / ", max(pos[:,2]))
    print ("box", boxSize)
    
    ## build the tree for the particles in this file:
    tree = 0
    tree = spatial.cKDTree(pos, boxsize=boxSize)
    print ("Tree built.")
    
    ## look at shells at fixed radiius

    LG_NP_file_kpc = []
    
    for r_bin, max_r in enumerate(shell_radii_kpc):
        
        LG_NP_shell = []
        
        #  Loop over the LG candidates and count the number of DM particles within the separation from the centre.
        for i in range(N_LG):
            NeighbourList = tree.query_ball_point([LG_x[i], LG_y[i], LG_z[i]], max_r)
            # Number of particles in this file, in this shell, for this LG
            LG_NP_shell.append( len(NeighbourList) )
        # Number of particles in this file, in this shell, for all LGs
        LG_NP_shell = np.array(LG_NP_shell)

        # Number of particles in this file, for all shells & for all LGs.
        LG_NP_file_kpc.append(LG_NP_shell)

    LG_NP_file_kpc = np.array(LG_NP_file_kpc)
    LG_NP_kpc = LG_NP_kpc + LG_NP_file_kpc.T
    
    ## look at shells at fixed comoving radiius

    LG_NP_file_com = []
    
    for r_bin, max_r in enumerate(shell_radii_com):
        
        LG_NP_shell = []
        
        #  Loop over the LG candidates and count the number of DM particles within the separation from the centre.
        for i in range(N_LG):
            NeighbourList = tree.query_ball_point([LG_x[i], LG_y[i], LG_z[i]], max_r)
            # Number of particles in this file, in this shell, for this LG
            LG_NP_shell.append( len(NeighbourList) )
        # Number of particles in this file, in this shell, for all LGs
        LG_NP_shell = np.array(LG_NP_shell)

        # Number of particles in this file, for all shells & for all LGs.
        LG_NP_file_com.append(LG_NP_shell)

    LG_NP_file_com = np.array(LG_NP_file_com)
    LG_NP_com = LG_NP_com + LG_NP_file_com.T
    
    
    ## look at shells as a fraction of separation
            
    LG_NP_file_r = []
    
    for r_bin, max_r in enumerate(shell_radii_r):
        
        LG_NP_shell = []
        
        #  Loop over the LG candidates and count the number of DM particles within the separation from the centre.
        for i in range(N_LG):
            max_r_LG = min(5000, max_r * LG_r[i])
            NeighbourList = tree.query_ball_point([LG_x[i], LG_y[i], LG_z[i]], max_r_LG)
            # Number of particles in this file, in this shell, for this LG
            LG_NP_shell.append( len(NeighbourList) )
        # Number of particles in this file, in this shell, for all LGs
        LG_NP_shell = np.array(LG_NP_shell)

        # Number of particles in this file, for all shells & for all LGs.
        LG_NP_file_r.append(LG_NP_shell)

    LG_NP_file_r = np.array(LG_NP_file_r)
    LG_NP_r = LG_NP_r + LG_NP_file_r.T
    
    # total number of particles in LG groups:"
    print ("total number of particles at fixed distance:", np.sum(LG_NP_kpc) )
    print ("total number of particles at fixed comoving distance:", np.sum(LG_NP_com) )
    print ("total number of particles at fractional distance:", np.sum(LG_NP_r) )

print ("done")

np.savez('MiniUchu_particledata_snap_'+str(snapshot)+'.npz', allow_pickle = True,
         shell_radii_kpc= shell_radii_kpc, shell_radii_r= shell_radii_r, shell_radii_com = shell_radii_com, 
         LG_NP_kpc=LG_NP_kpc, LG_NP_com = LG_NP_com, LG_NP_r = LG_NP_r)

print ("All Done")

file  1  of  400
s: <SimSnap "/scratch/pjohanss/sawalati/mini_uchuu_snapshots/MiniUchuu_015.gad" len=41129531>
len of s: 41129531
Memory used 175.31640625  MB
min / max x: 0.0003153894  /  6290.3994
min / max y: 5.2982017e-05  /  6054.824
min / max z: 0.0  /  128076.76
box 128076.77409802184
Tree built.
total number of particles at fixed distance: 15848655
total number of particles at fixed comoving distance: 443475
total number of particles at fractional distance: 4245710
file  2  of  400
s: <SimSnap "/scratch/pjohanss/sawalati/mini_uchuu_snapshots/MiniUchuu_015.gad.1" len=41682460>
len of s: 41682460
Memory used 2957.078125  MB
min / max x: 8.033521e-05  /  6290.3994
min / max y: 6054.824  /  12484.147
min / max z: 0.0  /  128076.76
box 128076.77409802184
Tree built.
total number of particles at fixed distance: 44262945
total number of particles at fixed comoving distance: 1123213
total number of particles at fractional distance: 11144065
file  3  of  400
s: <SimSnap "/scratch/pjohan

s: <SimSnap "/scratch/pjohanss/sawalati/mini_uchuu_snapshots/MiniUchuu_015.gad.17" len=42039719>
len of s: 42039719
Memory used 4419.80078125  MB
min / max x: 3.135162e-05  /  6290.3994
min / max y: 108528.984  /  114838.164
min / max z: 0.0054442356  /  128076.76
box 128076.77409802184
Tree built.
total number of particles at fixed distance: 576786582
total number of particles at fixed comoving distance: 15264595
total number of particles at fractional distance: 161943280
file  19  of  400
s: <SimSnap "/scratch/pjohanss/sawalati/mini_uchuu_snapshots/MiniUchuu_015.gad.18" len=42130477>
len of s: 42130477
Memory used 4928.69140625  MB
min / max x: 1.0530529e-05  /  6290.3994
min / max y: 114838.164  /  121762.07
min / max z: 0.015998898  /  128076.76
box 128076.77409802184
Tree built.
total number of particles at fixed distance: 613367978
total number of particles at fixed comoving distance: 16163770
total number of particles at fractional distance: 172910202
file  20  of  400
s: <SimSn

min / max x: 6290.3994  /  12791.83
min / max y: 89503.82  /  95884.49
min / max z: 0.0  /  128076.75
box 128076.77409802184
Tree built.
total number of particles at fixed distance: 1160739568
total number of particles at fixed comoving distance: 31468275
total number of particles at fractional distance: 318794746
file  36  of  400
s: <SimSnap "/scratch/pjohanss/sawalati/mini_uchuu_snapshots/MiniUchuu_015.gad.35" len=41501456>
len of s: 41501456
Memory used 4968.484375  MB
min / max x: 6290.4  /  12791.83
min / max y: 95884.49  /  102097.46
min / max z: 0.0  /  128076.76
box 128076.77409802184
Tree built.
total number of particles at fixed distance: 1174464599
total number of particles at fixed comoving distance: 31816777
total number of particles at fractional distance: 322225118
file  37  of  400
s: <SimSnap "/scratch/pjohanss/sawalati/mini_uchuu_snapshots/MiniUchuu_015.gad.36" len=41636779>
len of s: 41636779
Memory used 5421.45703125  MB
min / max x: 6290.3994  /  12791.83
min / ma

min / max x: 12791.83  /  19450.7
min / max y: 70290.71  /  76959.234
min / max z: 0.00016151022  /  128076.76
box 128076.77409802184
Tree built.
total number of particles at fixed distance: 1736886311
total number of particles at fixed comoving distance: 46795956
total number of particles at fractional distance: 472192836
file  53  of  400
s: <SimSnap "/scratch/pjohanss/sawalati/mini_uchuu_snapshots/MiniUchuu_015.gad.52" len=42507494>
len of s: 42507494
Memory used 6495.30078125  MB
min / max x: 12791.83  /  19450.7
min / max y: 76959.234  /  83193.6
min / max z: 0.0  /  128076.76
box 128076.77409802184
Tree built.
total number of particles at fixed distance: 1765855362
total number of particles at fixed comoving distance: 47665941
total number of particles at fractional distance: 478408938
file  54  of  400
s: <SimSnap "/scratch/pjohanss/sawalati/mini_uchuu_snapshots/MiniUchuu_015.gad.53" len=42245789>
len of s: 42245789
Memory used 6956.703125  MB
min / max x: 12791.83  /  19450.7
m

min / max y: 51164.86  /  57749.234
min / max z: 0.0  /  128076.76
box 128076.77409802184
Tree built.
total number of particles at fixed distance: 2360078574
total number of particles at fixed comoving distance: 63434072
total number of particles at fractional distance: 641798919
file  70  of  400
s: <SimSnap "/scratch/pjohanss/sawalati/mini_uchuu_snapshots/MiniUchuu_015.gad.69" len=41961393>
len of s: 41961393
Memory used 8359.1796875  MB
min / max x: 19450.7  /  25980.39
min / max y: 57749.234  /  64315.67
min / max z: 0.0  /  128076.76
box 128076.77409802184
Tree built.
total number of particles at fixed distance: 2408370368
total number of particles at fixed comoving distance: 64845378
total number of particles at fractional distance: 659302326
file  71  of  400
s: <SimSnap "/scratch/pjohanss/sawalati/mini_uchuu_snapshots/MiniUchuu_015.gad.70" len=41825159>
len of s: 41825159
Memory used 8838.9296875  MB
min / max x: 19450.7  /  25980.39
min / max y: 64315.67  /  70652.06
min / max

...

Memory used 9761.0234375  MB
min / max x: 102403.35  /  108826.49
min / max y: 19205.72  /  25403.473
min / max z: 0.0087054875  /  128076.76
box 128076.77409802184
Tree built.
total number of particles at fixed distance: 11209538573
total number of particles at fixed comoving distance: 303272152
total number of particles at fractional distance: 3110954824
file  325  of  400
s: <SimSnap "/scratch/pjohanss/sawalati/mini_uchuu_snapshots/MiniUchuu_015.gad.324" len=42197553>
len of s: 42197553
Memory used 7827.0546875  MB
min / max x: 102403.35  /  108826.49
min / max y: 25403.473  /  31661.549
min / max z: 0.0  /  128076.75
box 128076.77409802184
Tree built.
total number of particles at fixed distance: 11247090845
total number of particles at fixed comoving distance: 304150854
total number of particles at fractional distance: 3121205658
file  326  of  400
s: <SimSnap "/scratch/pjohanss/sawalati/mini_uchuu_snapshots/MiniUchuu_015.gad.325" len=41568143>
len of s: 41568143
Memory used 8313.8

min / max x: 108826.49  /  115385.44
min / max y: 8.6503605e-05  /  6492.267
min / max z: 0.0  /  128076.76
box 128076.77409802184
Tree built.
total number of particles at fixed distance: 11775891249
total number of particles at fixed comoving distance: 318720196
total number of particles at fractional distance: 3277136051
file  342  of  400
s: <SimSnap "/scratch/pjohanss/sawalati/mini_uchuu_snapshots/MiniUchuu_015.gad.341" len=42838841>
len of s: 42838841
Memory used 9758.05859375  MB
min / max x: 108826.49  /  115385.44
min / max y: 6492.267  /  12824.418
min / max z: 0.0  /  128076.76
box 128076.77409802184
Tree built.
total number of particles at fixed distance: 11806056956
total number of particles at fixed comoving distance: 319584164
total number of particles at fractional distance: 3283993180
file  343  of  400
s: <SimSnap "/scratch/pjohanss/sawalati/mini_uchuu_snapshots/MiniUchuu_015.gad.342" len=42503929>
len of s: 42503929
Memory used 10276.78515625  MB
min / max x: 108826.4

min / max x: 108826.49  /  115385.44
min / max y: 108580.24  /  115108.45
min / max z: 0.0  /  128076.76
box 128076.77409802184
Tree built.
total number of particles at fixed distance: 12448718459
total number of particles at fixed comoving distance: 337048302
total number of particles at fractional distance: 3464917367
file  359  of  400
s: <SimSnap "/scratch/pjohanss/sawalati/mini_uchuu_snapshots/MiniUchuu_015.gad.358" len=42085654>
len of s: 42085654
Memory used 5945.0234375  MB
min / max x: 108826.49  /  115385.44
min / max y: 115108.45  /  121804.11
min / max z: 0.0  /  128076.76
box 128076.77409802184
Tree built.
total number of particles at fixed distance: 12489834182
total number of particles at fixed comoving distance: 338063999
total number of particles at fractional distance: 3476447389
file  360  of  400
s: <SimSnap "/scratch/pjohanss/sawalati/mini_uchuu_snapshots/MiniUchuu_015.gad.359" len=42150778>
len of s: 42150778
Memory used 6400.3359375  MB
min / max x: 108826.49  / 

min / max x: 115385.44  /  121673.67
min / max y: 88909.516  /  95513.414
min / max z: 0.000992639  /  128076.76
box 128076.77409802184
Tree built.
total number of particles at fixed distance: 13038755748
total number of particles at fixed comoving distance: 352737627
total number of particles at fractional distance: 3628151494
file  376  of  400
s: <SimSnap "/scratch/pjohanss/sawalati/mini_uchuu_snapshots/MiniUchuu_015.gad.375" len=41930941>
len of s: 41930941
Memory used 7896.27734375  MB
min / max x: 115385.44  /  121673.67
min / max y: 95513.414  /  101844.02
min / max z: 0.0  /  128076.76
box 128076.77409802184
Tree built.
total number of particles at fixed distance: 13066900209
total number of particles at fixed comoving distance: 353483847
total number of particles at fractional distance: 3641332739
file  377  of  400
s: <SimSnap "/scratch/pjohanss/sawalati/mini_uchuu_snapshots/MiniUchuu_015.gad.376" len=41852439>
len of s: 41852439
Memory used 8334.9921875  MB
min / max x: 1153

min / max x: 0.0  /  128076.76
min / max y: 71229.62  /  76837.31
min / max z: 0.0035679685  /  128076.76
box 128076.77409802184
Tree built.
total number of particles at fixed distance: 13564845275
total number of particles at fixed comoving distance: 366485775
total number of particles at fractional distance: 3768719564
file  393  of  400
s: <SimSnap "/scratch/pjohanss/sawalati/mini_uchuu_snapshots/MiniUchuu_015.gad.392" len=41730021>
len of s: 41730021
Memory used 9260.65625  MB
min / max x: 0.0  /  128076.76
min / max y: 76837.31  /  82903.445
min / max z: 0.0  /  128076.76
box 128076.77409802184
Tree built.
total number of particles at fixed distance: 13596400794
total number of particles at fixed comoving distance: 367256979
total number of particles at fractional distance: 3776448348
file  394  of  400
s: <SimSnap "/scratch/pjohanss/sawalati/mini_uchuu_snapshots/MiniUchuu_015.gad.393" len=41874708>
len of s: 41874708
Memory used 9754.35546875  MB
min / max x: 0.0  /  128076.76
mi